In [5]:
import cv2
import numpy as np
import yaml
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets.samples_generator import make_blobs
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

In [6]:
frame = cv2.imread("frame_0.png", cv2.IMREAD_COLOR)
frame_size = (frame.shape[1], frame.shape[0])
#frame = cv2.resize(frame,(512,256), interpolation=cv2.INTER_LINEAR)
#frame = frame/127.5 -1
#cv2.imshow('blended.png', frame)
#cv2.waitKey(0)
print (len(frame.shape))

3


In [7]:
#Coordinates of quadrangle vertices in the source image.
src = np.float32([
                [450,230],
                [600,312],
                [80,312],
                [281,230]])

#Coordinates of the corresponding quadrangle vertices in the destination image.
dst = np.float32([
        [450,230],
        [450,312],
        [281,312],
        [281,230]])

In [8]:
H = cv2.getPerspectiveTransform(src, dst)
H_inv = cv2.getPerspectiveTransform(dst,src)
warped = cv2.warpPerspective(frame, H, frame_size, flags=cv2.INTER_LINEAR)
#cv2.imshow('blended.png', warped)
#cv2.waitKey()
#cv2.destroyAllWindows()
print(H)
#im1 = im1.save("geeks.jpg")

[[-2.07231726e-01 -1.98289269e+00  4.56065319e+02]
 [-5.68434189e-16 -1.84486781e+00  3.76656299e+02]
 [-2.63805169e-18 -5.24883359e-03  1.00000000e+00]]


In [9]:
# create indices of the destination image and linearize them
h, w = warped.shape[:2]
indy, indx = np.indices((h, w), dtype=np.float32)
lin_homg_ind = np.array([indx.ravel(), indy.ravel(), np.ones_like(indx).ravel()])

In [10]:
map_ind = H_inv.dot(lin_homg_ind)
map_x, map_y = map_ind[:-1]/map_ind[-1]
map_x = map_x.reshape(h, w).astype(np.float32) 
map_y = map_y.reshape(h, w).astype(np.float32)

In [11]:
# remap!
dst = cv2.remap(warped, map_x, map_y, cv2.INTER_LINEAR)
blended = cv2.addWeighted(frame, 0.5, dst, 0.5, 0)
#cv2.imshow('blended.png', blended)
#cv2.waitKey()
#cv2.destroyAllWindows()

In [12]:
fs_write = cv2.FileStorage('remap.yml', cv2.FILE_STORAGE_WRITE)

fs_write.write("remap_ipm_x", map_x)
fs_write.write("remap_ipm_y",map_y)
fs_write.release()

In [13]:
#ipm_remap_file_path ='./tusimple_ipm_remap.yml'
ipm_remap_file_path ='./remap.yml'
print (ipm_remap_file_path)

./remap.yml


In [14]:
_ipm_remap_file_path = ipm_remap_file_path

In [15]:
fs = cv2.FileStorage(_ipm_remap_file_path, cv2.FILE_STORAGE_READ)

remap_to_ipm_x = fs.getNode('remap_ipm_x').mat()
remap_to_ipm_y = fs.getNode('remap_ipm_y').mat()



In [ ]:
image_1 = cv2.remap(
                frame,
                remap_to_ipm_x,
                remap_to_ipm_y,
                interpolation=cv2.INTER_NEAREST)
print (image_1.shape)
cv2.imshow('blended.png', image_1)
cv2.waitKey(0)

(480, 640, 3)
